# Transportation Simulation using Agent Based approach 

In this version, no pre-simulation library used, all simulation approches hard coded from scratch.

In [ ]:
! pip install googlemaps
! pip install DateTime
! pip install gmplot
! pip install bs4

In [18]:
import pandas as pd
import random  #import randrange
import datetime as dt
#Googel API code
import googlemaps
import urllib
import numpy as np
import sys
import urllib, json
import time
import threading
from gmplot import gmplot
import os
from bs4 import BeautifulSoup
from sys import platform


Add your Google Map API key here

In [2]:
KEY = ''# Google map API key 


In [3]:
folder=''# inset your databse directory 
gmaps = googlemaps.Client(KEY)  # google API key
dataset = pd.read_csv('postcodes.csv')   #place your postcode datasets

# Importing the dataset
dataset = pd.read_csv(folder+'postcodes.csv')
dataset = dataset[dataset['In Use?']=='Yes']
postcode=list(dataset['Postcode'])
#demografic= pd.read_csv("filename.csv")
sim_date=[2019, 12, 15] #year,month,day

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### rules: 

**h_shift** is the hours after zero_time that one's day starts
**min_var** is minut variation from the nominal start day time
**start** is date time with this format (2018, 12, 20,8,00)
**timedelta** (weeks=40, days=84, hours=23,minutes=50, seconds=600)  # adds up to 365 days  

In [4]:
def random_time(start,h_shift,min_var): 
        date_time = start + dt.timedelta(hours=h_shift,minutes=random.randrange(min_var),seconds=0) #timedelta(weeks=40, days=84, hours=23,minutes=50, seconds=600)  # adds up to 365 days       
        #print (date_time.strftime("%d/%m/%y %H:%M"))
        #print (date_time.strftime("%H:%M:%S"))
        return date_time
    



## Google map places_finder


In [5]:
def places_finder(where,radius,types):
    #making the url
    geo= gmaps.geocode(str(where))
    latlng=geo[0]['geometry']['bounds']['northeast']
    
    LOCATION = str(latlng['lat']) + "," + str(latlng['lng'])
    RADIUS = radius
    TYPES = types
    MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
            '?location=%s'
            '&radius=%s'
            '&types=%s'
            '&sensor=false&key=%s') % (LOCATION, RADIUS, TYPES, KEY)
    #grabbing the JSON result
    response = urllib.request.urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw.decode())
    location=jsonData['results'][0]['geometry']['location']
    return  str(location['lat'])+','+str(location['lng'])
  
  

In [6]:
#ref: https://stackoverflow.com/questions/15380712/how-to-decode-polylines-from-google-maps-direction-api-in-php
import geopy.distance
def decode_polyline(polyline_str):
    index, lat, lng = 0, 0, 0
    coordinates = []
    dists=[0]
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))
        if len(coordinates)>1:
            dists.append(geopy.distance.distance(coordinates[-1], coordinates[-2]).m) #returns the distance in 'm' , replace it with 'km' if needed

    return coordinates, dists


In [7]:
class Spinner:
    busy = False
    delay = 0.1
    @staticmethod
    def spinning_cursor():
        while 1: 
            for cursor in '|/-\\': yield cursor
    def __init__(self, delay=None):
        self.spinner_generator = self.spinning_cursor()
        if delay and float(delay): self.delay = delay
    def spinner_task(self):
        while self.busy:
            sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b'+next(self.spinner_generator)+' WORKING...')
            sys.stdout.flush()
            time.sleep(self.delay)
    def start(self):
        self.busy = True
        threading.Thread(target=self.spinner_task).start()
    def stop(self):
        self.busy = False
        time.sleep(self.delay)        
        sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b'+'FINISHED!')
         


### MAIN CLASSES


In [8]:
class make_individual:
    def __init__(self,home_adrs,gender,age,social_cls,house_hdcount,work_adrs,cars=None,job_cls=None,n_child=None,child_ages=[]):#child ages is a list of all childeren in each house,which effects on trip type to nursery or school
        
        self.home_adrs=home_adrs
        self.gender=gender
        self.age=age
        self.social_cls=social_cls
        self.house_hdcount=house_hdcount
        self.cars=cars
        self.work_adrs=work_adrs
        self.job_cls=job_cls
        self.n_child=n_child
        self.child_ages=child_ages
        
        # additional places:
        try:
            self.gym_adrs= places_finder(self.home_adrs,5000,'gym')
        except:self.gym_adrs= places_finder(self.work_adrs,5000,'gym')
        try:
            self.nursery_adrs= places_finder(self.home_adrs,5000,'nursery')
        except:self.nursery_adrs= places_finder(self.work_adrs,5000,'nursery')
        try:
            self.shop_adrs= places_finder(self.home_adrs,5000,'supermarket')
        except: self.shop_adrs= places_finder(self.work_adrs,5000,'supermarket')

            


    def trips(self):
        zero_time = dt.datetime(sim_date[0],sim_date[1],sim_date[2],00,00,00)   
        self.trip_nodes={'Single_people':[
                        ['HOME','WORK','HOME'],
                        ['HOME','WORK','GYM','HOME'],
                        ['HOME','WORK','SHOP','HOME'],
                        ['HOME','WORK','GYM','SHOP','HOME'],
                        ['HOME','WORK','SHOP','GYM','HOME'],
                        ['HOME','GYM','WORK','HOME'],
                        ['HOME','GYM','WORK','SHOP','HOME']] 
                       }
        self.trip_nodes={'Single_people':[['HOME','WORK','HOME']]}   
        
        if self.work_adrs!=None and self.house_hdcount==1 : 
            steps=random.choice(self.trip_nodes['Single_people'])
        self.trips=[]
        for i in range(len(steps)-1):
            self.trips.append(steps[0+i:2+i])
            # before work activities
            if self.trips[i]==['HOME','GYM'] : self.trips[i].append('leave');self.trips[i].append(random_time(zero_time,6,40))# 6 hrs and +-~40min shift from zero
            elif self.trips[i][1]=='WORK' :self.trips[i].append('arrive');self.trips[i].append(random_time(zero_time,8,45))
            # after work activities : they are timed based on last activity time (trips[-1][-1])
            
            elif self.trips[i][0]=='WORK' :self.trips[i].append('leave');self.trips[i].append(random_time(self.trips[-2][-1],8,30)) #time spend at work: 8h
            elif self.trips[i][0]=='GYM'  :self.trips[i].append('leave');self.trips[i].append(random_time(self.trips[-2][-1],1,30))#time spend at gym: 1h
            elif self.trips[i][0]=='SHOP' :self.trips[i].append('leave');self.trips[i].append(random_time(self.trips[-2][-1],0,50)) 
            else : pass
        #print (self.trips)
        
        #assigning actual addresses to the trip lists:
        actual_places = {'HOME':self.home_adrs, 'WORK':self.work_adrs, 'GYM':self.gym_adrs, 'SHOP':self.shop_adrs}
        for i in range(len(self.trips)):
            self.trips[i]=[actual_places.get(n, n) for n in self.trips[i]]
        return self.trips
    
    

    #route maker                        
    def route(self,trip_legs):
        routes=[]
        for leg in trip_legs:
            #leg[0],leg[1] =origini and destination of each leg,leg[2]is departure or arrival, leg[-1] is the time
            if leg[2]=='leave':
                directions_result = gmaps.directions(leg[0], leg[1], mode="driving", departure_time=leg[-1])
            else: directions_result = gmaps.directions(leg[0],leg[1],mode="driving",arrival_time=leg[-1])                                 
            #extracting the key infor ployline points from a direcion_results 
            route_info=directions_result[0]['legs'][0] 
            #key info : start time, start location, total trip disttance, total trip time ,end location, end time
            if leg[2]=='leave':
                start_time=leg[-1];
                end_time=leg[-1] + dt.timedelta(seconds=route_info['duration']['value'])#this calculates the end time if we know the leave time

            else: 
                start_time=leg[-1] - dt.timedelta(seconds=route_info['duration']['value']) #this calculates the start time if we know the arrival time
                end_time=leg[-1]
        
            leg_key_info={'start_time':start_time,
                                'start_location':route_info['start_location'],
                                'distance':route_info['distance']['value'],
                                'duration':route_info['duration']['value'],
                                'end_location':route_info['end_location'],
                                'end_time':end_time
                            }   
            #extracting the ployline points from a route                                                                                                         
            routes.append({'leg_key_info':leg_key_info,'steps':route_info['steps']})
        #the route is a list of trip route/info for each leg, for instance [0] returns 1st leg and will containe below dictionaries: 'leg_key_info' and 'steps'
        '''
        {'leg_key_info': 
        {'distance': 114392,
        'duration': 5205,
        'end_location': {'lat': 51.7510628, 'lng': -1.2583664},
        'end_time': datetime.datetime(2018, 12, 20, 8, 10),
        'start_location': {'lat': 51.9067391, 'lng': -0.2078139},
        'start_time': datetime.datetime(2018, 12, 20, 6, 43, 15)},
        'steps':
        [{'distance': {'text': '0.2 km', 'value': 213},
        'duration': {'text': '1 min', 'value': 20},
        'end_location': {'lat': 51.9049052, 'lng': -0.2069715},
        'html_instructions': 'Head <b>south</b> on <b>Lytton Way</b>/<b>A602</b> toward <b>Gates Way</b>',
        'polyline': {'points': 'c`y{Hxqg@|@YxAg@\\KvCw@JIRW'},
        'start_location': {'lat': 51.9067391, 'lng': -0.2078139},
        'travel_mode': 'DRIVING'},
        ''' 
        return routes   
    
    def loc8(self,t): #location of agent at time = t (datetime.datetime type)

        for l in self.route(self.trips):
            if  t <= l['leg_key_info']['start_time']: 
                loc=(l['leg_key_info']['start_location']['lat'],l['leg_key_info']['start_location']['lng'])
                return loc #(loc[1],loc[0])
                break
                

            elif l['leg_key_info']['start_time'] < t <= l['leg_key_info']['end_time']:
                t=t-l['leg_key_info']['start_time']
                t=int(t.total_seconds())
                i=0
                while t>l['steps'][i]['duration']['value']:
                    t=t-l['steps'][i]['duration']['value']
                    i+=1 #this 'i' once out of the shows in which step of the leg the agent is 
                #making a displacement vector from start position (ps) and end position (pe)

                step_points,step_length=decode_polyline(l['steps'][i]['polyline']['points'])#**************************
                step_velocity=l['steps'][i]['distance']['value']/l['steps'][i]['duration']['value']
                step_time=np.array(step_length)/step_velocity # this list 'step_time' starts with '0' as first item
                ii=0
                while ii+1<len(step_time) and t>step_time[ii+1]:
                    t=t-step_time[ii+1]
                    ii+=1
                pe=step_points[ii+1]
                ps=step_points[ii]
                vec=np.array(pe)-np.array(ps)
                #making a partial displacement vector based on the time passed from start position
                tt=t/step_time[ii+1]
                vec=tt*vec
                #current position : ps + partial vector
                loc=tuple(ps+vec)# array
                return loc
                break
            else: 
                loc=(l['leg_key_info']['end_location']['lat'],l['leg_key_info']['end_location']['lng']);
                return loc        
            
   
    

#### Test

In [9]:
h=random.choice(postcode)  #home postcode
w=random.choice(postcode)  #work postcode
p1=make_individual(h,'male',23,'fdfdf',1,w)
p1.trips()    #///second time without ()
#use below to see all the steps and timings:
q=p1.route(p1.trips)
for l in q:
    print ('start->: ', l['leg_key_info']['start_location'], '@',l['leg_key_info']['start_time'])
    print ('--->end: ', l['leg_key_info']['end_location'], '@',l['leg_key_info']['end_time'])
    print('---------------------------------------------------------------------------------')
#use below to loc8 the agent at different times:
print(p1.loc8(dt.datetime(2019,8,8,12,50))) #try different times )

start->:  {'lat': 51.60594889999999, 'lng': -1.2351898} @ 2019-12-15 07:54:37
--->end:  {'lat': 51.7543967, 'lng': -1.2191121} @ 2019-12-15 08:25:00
---------------------------------------------------------------------------------
start->:  {'lat': 51.7543967, 'lng': -1.2191121} @ 2019-12-15 16:51:00
--->end:  {'lat': 51.60594889999999, 'lng': -1.2351898} @ 2019-12-15 17:21:01
---------------------------------------------------------------------------------
(51.60594889999999, -1.2351898)


# SIMULATION


Define some rules: 

In [10]:

sim_date=[2019, 9, 20] #year,month,day DATE of simulation
t_start=dt.datetime(sim_date[0],sim_date[1],sim_date[2], 7,0)# start time of simulation
t_end=dt.datetime(sim_date[0],sim_date[1],sim_date[2], 9,0)# end time of simulation
time_resolution=60 #seconds


## Mutil Agents at certain time of the day


In [12]:
population=50  #set the city population here. for fast run, I picked 50 here

LOCs=[[]]*population
time1=dt.datetime.now()
for i in range(population):
    h=random.choice(postcode)
    w=random.choice(postcode)  
    p1=make_individual(h,'male',23,'fdfdf',1,w)
    p1.trips()
    #q=p1.route(p1.trips)
    LOCs[i]=p1.loc8(t_start)

    sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+'Agent: '+str(1+i)+' / '+str(population))
    sys.stdout.flush()
print ('\n',dt.datetime.now()-time1)

Agent: 1 / 5Agent: 2 / 5Agent: 3 / 5Agent: 4 / 5Agent: 5 / 5Agent: 6 / 5Agent: 7 / 5Agent: 8 / 5Agent: 9 / 5Agent: 10 / 5Agent: 11 / 5Agent: 12 / 5Agent: 13 / 5Agent: 14 / 5Agent: 15 / 5Agent: 16 / 5Agent: 17 / 5Agent: 18 / 5Agent: 19 / 5Agent: 20 / 5Agent: 21 / 5Agent: 22 / 5Agent: 23 / 5Agent: 24 / 5Agent: 25 / 5Agent: 26 / 5Agent: 27 / 5Agent: 28 / 5Agent: 29 / 5Agent: 30 / 5Agent: 31 / 5Agent: 32 / 5Agent: 33 / 5Agent: 34 / 5Agent: 35 / 5Agent: 36 / 5Agent: 37 / 5Agent: 38 / 5Agent: 39 / 5Agent: 40 / 5Agent: 41 / 5Agent: 42 / 5Agent: 43 / 5Agent: 44 / 5Agent: 45 / 5Agent: 46 / 5Agent: 47 / 5Agent: 48 / 5Agent: 49 / 5Agent: 50 / 50
 0:02:01.832219


## One Agent followed at different times


In [13]:
spinner = Spinner()
spinner.start()
LOCs=[]
h=random.choice(postcode)
w=random.choice(postcode)  
p1=make_individual(h,'male',23,'fdfdf',1,w)
p1.trips()
t=t_start
while t<t_end :
    LOCs.append(p1.loc8(t))
    t = t + dt.timedelta(seconds=time_resolution)
spinner.stop()



| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.| WORKING./ WORKING.- WORKING.\ WORKING.

## Visualization

In [19]:
def insertapikey(fname, apikey):
    """put the google api key in a html file"""
    def putkey(htmltxt, apikey, apistring=None):
        """put the apikey in the htmltxt and return soup"""
        if not apistring:
            apistring = "https://maps.googleapis.com/maps/api/js?key=%s&callback=initMap"
        soup = BeautifulSoup(htmltxt, 'html.parser')
        body = soup.body
        src = apistring % (apikey, )
        tscript = soup.new_tag("script", src=src)#, async="defer")
        body.insert(-1, tscript)
        return soup
    htmltxt = open(fname, 'r').read()
    soup = putkey(htmltxt, apikey)
    newtxt = soup.prettify()
    open(fname, 'w').write(newtxt)

    
gmap = gmplot.GoogleMapPlotter(51.827922, -0.790973, 11)
p_lat,p_lng= zip(*LOCs)
#gmap.marker(51.827922, -0.790973, 'cornflowerblue')
gmap.scatter(p_lat, p_lng, 'red', size=200, marker=False)
# Draw
#gmap.plot(p_lat, p_lng, 'red')
map_file=folder+"map1.html" #spide mac and windows 

gmap.draw(map_file)
insertapikey(map_file, KEY)


if platform == "darwin": os.system("open "+map_file)# OS X
elif platform == "win32": os.system("start "+map_file)#Windows
else: pass

In [21]:
######## the path og agent movement will apear in a new web page on the region of map you defined start and end point.

In [22]:
os.system("open "+map_file)

0